<a href="https://colab.research.google.com/github/kpmadabhu/EEE598_proj_tuh_eeg/blob/main/EEE598_Proj_TUH_EEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TUSZ Preprocessing Pipeline

This notebook builds a seizure vs background window dataset from TUH Seizure Corpus (TUSZ) tcp-bipolar montage EEG.

- Recursively walks `train/`, `dev/`, or `eval` under `ROOT_DIR`.
- For each EDF:
  - Load EEG + its seizure annotations (.csv).
  - Bandpass 0.5–40 Hz using a Kaiser FIR, zero-phase (`filtfilt`).
  - Downsample to 250 Hz *if* original fs is higher; never upsample.
  - Robust z-score per channel (median/IQR).
  - Slice into 10 s windows with 5 s hop.
  - Label each window seizure(1) or background(0), pooling all seizure subtypes.
- Skips recordings that are too short for `filtfilt` padding.
- Saves:
  - `split_windows.npz` with windows, labels, and metadata.
  - `qc_window.png` plotting an informative example window.
  - `class_balance.png` showing seizure/background counts.

## Smart Canonical Montage
We do **not** force a giant hard-coded canonical bipolar channel list upfront.
Instead we learn a canonical layout from *inside the split*:

1. The **first usable recording** becomes our **reference layout**.
   - We store its channel names/ordering. Call this `ref_ch_names`.
2. Every later recording is aligned to that same reference layout:
   - For each reference channel name (like `FP1-F7`), we try to find the best-matching channel name in the new recording using a robust normalizer.
   - If found, we copy that channel.
   - If missing, we fill zeros in that row.
3. We compute a **coverage ratio** = fraction of reference channels we could actually fill with real data from this recording.
   - If coverage is too low (default <30%), we SKIP that recording to avoid mostly-zero data.


## Runtime knobs
- `MAX_SUBJECTS`: cap how many subjects to include (fast debug).
- `MAX_RECORDINGS`: cap recordings per subject (or overall).
- `NUM_TAPS`: FIR length. Shorter => fewer clips skipped for being "too short".
- `GROUP_MODE`: iterate by `"subject"` or just all EDFs (`"recording"`).
- `min_coverage`: skip a recording if < this fraction of reference channels could be mapped.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import mne
from scipy.signal import firwin, filtfilt, resample_poly
from math import gcd
import io, re
import random

# ------------------ USER CONFIG ------------------
# Point this at the root of the TUSZ split folders that contain train/dev/eval
ROOT_DIR = Path("Enter the mounted onedrive location of the edf folder of tusz database").expanduser().resolve()

# Output directory for NPZ + plots
OUT_DIR  = Path("output folder to save the windows ans coverage stats").expanduser().resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Windowing params
WIN_LEN_S = 10.0   # seconds per window
HOP_S     = 5.0    # seconds hop (50% overlap)

# DSP params
NUM_TAPS  = 1001    # Kaiser FIR taps for bandpass 0.5-40Hz (shorter -> fewer skips)
TARGET_FS = 250.0  # We'll downsample to 250 Hz if fs_orig > 250 Hz; never upsample

# Runtime limiting params
MAX_SUBJECTS   = 10        # limit number of subjects used (debug speed). None = no explicit cap
MAX_RECORDINGS = 10      # max recordings per subject / overall. None = no explicit cap
GROUP_MODE     = "subject" # "subject" or "recording"

# Seizure subtype labels we consider "seizure" (label=1)
# Everything else becomes background (label=0)
SEIZURE_LABELS = {
    'seiz','fnsz','gnsz','spsz','cpsz','absz',
    'tnsz','cnsz','tcsz','atsz','mysz'
}

# Artifact / noise labels in TUSZ annotations.
# Based on TUSZ event map, includes muscle, eye movement, chewing, shiver,
# electrode pops and combos
ARTIFACT_LABELS = {
    'artf',        # generic artifact
    'eyem',        # eye movement / blink
    'chew',        # chewing
    'shiv',        # shiver / tremor
    'musc',        # muscle artifact
    'elec',        # electrode pop / line noise
    # common multi-label combos found in TUSZ maps:
    'eyem_chew','eyem_shiv','eyem_musc','eyem_elec',
    'chew_shiv','chew_musc','chew_elec',
    'shiv_musc','shiv_elec',
    'musc_elec'
}

print("ROOT_DIR:", ROOT_DIR)
print("OUT_DIR:", OUT_DIR)


## Filtering / resampling / normalization

### Bandpass 0.5–40 Hz (Kaiser FIR)
- We design a linear-phase FIR using `firwin(..., window=('kaiser', beta))`.
- We run `filtfilt` per channel to get zero-phase.
- `filtfilt` needs enough padding, so we skip recordings that are too short.

### Resampling
- If original fs > 250 Hz, downsample to 250 using polyphase (`resample_poly`).
- If fs <= 250 Hz, leave it (no upsampling).

### Robust z-score
- Per channel: subtract median, divide by IQR.
- Flat channels (IQR ≈ 0) become ~0, which is fine.

In [ ]:
def _kaiser_beta_from_ripple(ripple_db: float) -> float:
    """
    Approximate Kaiser beta as a function of desired stopband attenuation in dB.
    """
    A = ripple_db
    if A > 50:
        return 0.1102 * (A - 8.7)
    elif A >= 21:
        return 0.5842 * (A - 21)**0.4 + 0.07886 * (A - 21)
    else:
        return 0.0

def apply_kaiser_bandpass(data, fs_orig, f_lo=0.5, f_hi=40.0, num_taps=101, ripple_db=60.0):
    n_ch, n_samp = data.shape
    padlen_needed = 3 * (num_taps - 1)
    if n_samp <= padlen_needed:
        raise ValueError(
            f"too short for filtfilt len={n_samp} need>{padlen_needed}"
        )

    beta = _kaiser_beta_from_ripple(ripple_db)
    taps = firwin(
        numtaps=num_taps,
        cutoff=[f_lo, f_hi],
        pass_zero=False,
        window=("kaiser", beta),
        fs=fs_orig,
        scale=True,
    )

    # zero-phase per channel
    filtered = np.stack([filtfilt(taps, [1.0], ch, axis=0) for ch in data], axis=0)
    return filtered

def maybe_resample_to_250(data, fs_orig, target_fs=TARGET_FS):
    if abs(fs_orig - target_fs) < 1e-6:
        return data, fs_orig
    if fs_orig < target_fs:
        print(f"WARNING: fs {fs_orig} Hz < {target_fs} Hz, not upsampling.")
        return data, fs_orig

    # integer ratio for resample_poly
    from math import gcd
    up = int(target_fs * 1000)
    down = int(fs_orig * 1000)
    g = gcd(up, down)
    up //= g
    down //= g

    resampled = []
    for ch in data:
        ch_rs = resample_poly(ch, up, down)
        resampled.append(ch_rs)
    resampled = np.stack(resampled, axis=0)
    return resampled, target_fs

def robust_zscore(data, eps=1e-9):
    med = np.median(data, axis=1, keepdims=True)
    p75 = np.percentile(data, 75, axis=1, keepdims=True)
    p25 = np.percentile(data, 25, axis=1, keepdims=True)
    iqr = p75 - p25
    return (data - med) / (iqr + eps)

## Annotation loader (CSV)

TUSZ seizure annotations are in `.csv` alongside the EDF. We:
1. Open the `.csv` and try to locate the header line that contains `start` and `stop`.
2. Read from that line forward using pandas.
3. Extract time intervals for any label in a target label set.

We create two label sets:
- `SEIZURE_LABELS`: all events we consider “seizure” for model supervision.
- `ARTIFACT_LABELS`: all events we consider “artifact / unusable background”, including muscle noise, eye movement, chewing, electrode pops, etc.

We then define:
- `load_seizure_intervals(csv_path)`:merged `(start_sec, stop_sec)` intervals containing seizure activity.
- `load_artifact_intervals(csv_path)`:merged `(start_sec, stop_sec)` intervals containing artifact.

Merging step:
- If two intervals overlap or touch, we merge them into a single continuous interval.  
  This prevents double-counting when annotations are dense.


These intervals are later used to decide if each 10 s window is seizure, clean background, or should be thrown away due to artifact.

In [ ]:
def _parse_annotation_csv(csv_path: Path):

    csv_path = Path(csv_path)
    try:
        raw_text = csv_path.read_text(errors="replace")
    except Exception as e:
        print(f"WARNING: Could not open {csv_path}: {e}")
        return None

    lines = raw_text.splitlines()
    header_idx = None
    for i, line in enumerate(lines):
        low = line.lower()
        if ("start" in low) and ("stop" in low):
            header_idx = i
            break

    if header_idx is None:
        # try naive pandas read
        try:
            ann = pd.read_csv(
                csv_path,
                sep=",",
                engine="python",
                on_bad_lines="skip"
            )
        except Exception as e:
            print(f"WARNING: Could not parse {csv_path}: {e}")
            return None
    else:
        main_body = "\n".join(lines[header_idx:])
        try:
            ann = pd.read_csv(
                io.StringIO(main_body),
                sep=",",
                engine="python",
                on_bad_lines="skip"
            )
        except Exception as e:
            print(f"WARNING: Could not smart-parse {csv_path}: {e}")
            return None

    if ann is None or ann.empty:
        return None

    # normalize column names
    ann.columns = [c.strip().lower() for c in ann.columns]
    return ann


def _extract_intervals_from_ann_df(ann_df: pd.DataFrame,
                                   target_label_set: set[str]):

    if ann_df is None or ann_df.empty:
        return []

    # figure out column names for start/stop and label
    if 'start_time' in ann_df.columns:
        start_col = 'start_time'
    elif 'start' in ann_df.columns:
        start_col = 'start'
    else:
        return []

    if 'stop_time' in ann_df.columns:
        stop_col = 'stop_time'
    elif 'stop' in ann_df.columns:
        stop_col = 'stop'
    else:
        return []

    if 'label' in ann_df.columns:
        label_col = 'label'
    elif 'type' in ann_df.columns:
        label_col = 'type'
    else:
        return []

    # normalize the label column
    tmp = ann_df.copy()
    tmp[label_col] = tmp[label_col].astype(str).str.lower().str.strip()

    use_rows = tmp[tmp[label_col].isin(target_label_set)]
    if use_rows.empty:
        return []

    raw_intervals = []
    for _, r in use_rows.iterrows():
        try:
            s = float(r[start_col])
            e = float(r[stop_col])
        except Exception:
            continue
        if not np.isfinite(s) or not np.isfinite(e):
            continue
        if e <= s:
            continue
        raw_intervals.append([s, e])

    if not raw_intervals:
        return []

    # merge overlaps
    raw_intervals.sort(key=lambda x: x[0])
    merged = [raw_intervals[0]]
    for s, e in raw_intervals[1:]:
        ls, le = merged[-1]
        if s <= le:
            merged[-1][1] = max(le, e)
        else:
            merged.append([s, e])

    return merged


def load_seizure_intervals(csv_path: Path):
    ann_df = _parse_annotation_csv(csv_path)
    return _extract_intervals_from_ann_df(ann_df, SEIZURE_LABELS)


def load_artifact_intervals(csv_path: Path):
    ann_df = _parse_annotation_csv(csv_path)
    return _extract_intervals_from_ann_df(ann_df, ARTIFACT_LABELS)

## Sliding window extraction and labeling

We cut the filtered, resampled, z-scored EEG into 10 s windows with 5 s hop.

For each window `[w_start, w_end]`:
1. If it overlaps any **seizure interval**: we KEEP the window and label it as seizure (`label = 1`).
   - Seizure **wins** over artifact. We do *not* discard seizure windows even if they contain some artifact. We want to keep seizure evidence.
2. Else if it overlaps any **artifact interval** (muscle, chewing, blink, electrode pop, etc.) -> we DROP that window entirely. It will *not* appear in the dataset.
   - This prevents noisy or motion-contaminated background from being learned as “normal.”
3. Else: we KEEP the window and label it as clean background (`label = 0`).

We return:
- `X_arr`: `[N_win, C, T]` (Number of windows, Channels and Time Samples per 10s window)
- `y_arr`: `[N_win]` seizure/background labels
- `t0_arr`: `[N_win]` start time (sec from file start)

In [ ]:
def window_data_with_artifact(data,
                              fs,
                              seizure_intervals,
                              artifact_intervals,
                              win_len_s,
                              hop_s):

    n_ch, n_samp = data.shape
    win_len = int(round(win_len_s * fs))
    hop     = int(round(hop_s * fs))

    X_list  = []
    Y_list  = []
    T0_list = []

    dropped_art = 0

    idx = 0
    while idx + win_len <= n_samp:
        seg = data[:, idx:idx + win_len]
        w_start = idx / fs
        w_end   = (idx + win_len) / fs

        # check seizure overlap
        is_seiz = any((w_start < sz_e) and (w_end > sz_s)
                      for (sz_s, sz_e) in seizure_intervals)

        if is_seiz:
            X_list.append(seg)
            Y_list.append(1)
            T0_list.append(w_start)
            idx += hop
            continue

        # check artifact overlap (only if not seizure)
        is_artf = any((w_start < a_e) and (w_end > a_s)
                      for (a_s, a_e) in artifact_intervals)

        if is_artf:
            # drop this window
            dropped_art += 1
            idx += hop
            continue

        # clean background
        X_list.append(seg)
        Y_list.append(0)
        T0_list.append(w_start)

        idx += hop

    if not X_list:
        return (
            np.empty((0, n_ch, win_len), dtype=np.float32),
            np.empty((0,), dtype=np.int64),
            np.empty((0,), dtype=np.float32),
            dropped_art
        )

    X_arr  = np.stack(X_list, axis=0).astype(np.float32)
    y_arr  = np.array(Y_list, dtype=np.int64)
    t0_arr = np.array(T0_list, dtype=np.float32)

    return X_arr, y_arr, t0_arr, dropped_art

## Matching EDF ↔ CSV

`file.edf` -> `file.csv`

In [ ]:
def find_best_annotation_for_edf(edf_path: Path):
    edf_path = Path(edf_path)
    d = edf_path.parent
    stem = edf_path.stem

    exact = d / f"{stem}.csv"
    if exact.exists():
        return exact

    csvs = list(d.glob('*.csv'))
    pref = [c for c in csvs if c.stem.startswith(stem)]
    if pref:
        return pref[0]

    if len(csvs) == 1:
        return csvs[0]

    return None

## Smart Canonical Montage: channel normalization + alignment

Normalize channel names so variants like:
- `EEG FP1-F7`
- `fp1_f7-ref`
- `FP1–F7`
map to the same key: `FP1-F7`.
This handles prefixes (`EEG `), suffixes (`-REF`, `-AVG`, etc.), underscores, dashes.

For each processed recording *after* filtering/z-scoring/windowing:
- We know its own channel names (`dbg['channel_names']`).
- We know the **reference channel layout** picked from the *first* good recording `ref_ch_names`.
- We build a lookup from normalized channel name -> row index in this recording.
- For each ref channel name, if we find a match, we copy that row; if not, we insert zeros.
- We track coverage_ratio = fraction of ref channels we successfully filled.
If coverage is too low (<30% by default), we skip that recording so we don't poison the dataset with mostly zeros.


In [ ]:
def _normalize_ch_name(name: str) -> str:

    n = name.upper().strip()

    # drop leading 'EEG '
    n = re.sub(r'^EEG\s+', '', n)

    # normalize fancy dashes to plain '-'
    n = n.replace('–', '-').replace('—', '-')

    # collapse spaces/underscores into '-'
    n = re.sub(r'[\s_]+', '-', n)

    # remove common reference suffixes like -REF, -AVG, -AV, -LE, A1/A2, M1/M2
    n = re.sub(r'-(REF|AVG|AV|LE|A1|A2|M1|M2)$', '', n)

    # collapse multiple dashes
    n = re.sub(r'-+', '-', n)

    # trim stray '-'
    n = n.strip('- ')

    return n

def align_to_reference(data_curr, ch_names_curr, ref_ch_names):

    norm_lookup = {}
    for i, nm in enumerate(ch_names_curr):
        norm_lookup[_normalize_ch_name(nm)] = i

    T = data_curr.shape[1]
    out = np.zeros((len(ref_ch_names), T), dtype=data_curr.dtype)

    filled = 0
    for i, ref_nm in enumerate(ref_ch_names):
        idx = norm_lookup.get(_normalize_ch_name(ref_nm))
        if idx is not None:
            out[i, :] = data_curr[idx, :]
            filled += 1

    coverage_ratio = filled / max(1, len(ref_ch_names))
    return out, coverage_ratio

## Preprocess a single EDF (Reference channel format)

Steps:
1. Load EDF with MNE.
2. 0.5–40 Hz bandpass via Kaiser FIR + zero-phase `filtfilt`.
3. Downsample to 250 Hz if fs > 250 (never upsample).
4. Robust z-score per channel.
5. Load seizure intervals from .csv, merge overlaps.
6. Slice into 10s windows / 5s hop.
7. Label seizure(1)/background(0).
8. We call `window_data_with_artifact(...)`, which slides 10 s / 5 s hop windows across the normalized EEG and returns only the windows we want to train on.
   - Rules:
     - Overlap seizure: KEEP (label=1).
     - Overlap artifact (but no seizure) -> DROP.
     - Otherwise: KEEP as background (label=0).
   - We count:
     - how many windows we kept,
     - how many were labeled seizure vs background,
     - how many artifact-only windows we dropped.

Returns:
- `X` shape `[N_win, C_curr, T]` in THE RECORDING'S OWN channel order
- `y` shape `[N_win]`
- `meta` dict with bookkeeping
- `debug_preview` dict with windows, labels, fs, channel_names (used later for alignment & QC)

In [ ]:
def preprocess_single_recording_raw(
    edf_path: Path,
    ann_csv_path: Path,
    win_len_s=WIN_LEN_S,
    hop_s=HOP_S,
    num_taps=NUM_TAPS
):


    print(f"INFO ONLY: EDF: {edf_path.as_posix()}")
    print(f"CSV: {ann_csv_path.as_posix()}")

    # -------------------------------------------------
    # Load raw EDF with MNE
    # -------------------------------------------------
    raw = mne.io.read_raw_edf(
        edf_path.as_posix(),
        preload=True,
        verbose='ERROR'
    )

    data = raw.get_data()      # shape [C_raw, N_samples]
    ch_names = raw.ch_names    # list of length C_raw
    fs_orig = float(raw.info['sfreq'])
    n_samp = data.shape[1]

    print(f"fs_orig={fs_orig} Hz  ch={data.shape[0]}  n_samples={n_samp}")

    '''
     Check if clip is long enough for filtfilt padding
     filtfilt needs ~3*(num_taps-1) samples of margin.
     If not long enough, we cannot safely apply the Kaiser filter.
     We just skip this recording entirely.
    '''
    padlen_needed = 3 * (num_taps - 1)
    if n_samp <= padlen_needed:
        print(f"SKIP SHORT: len={n_samp/fs_orig:.1f}s "
              f"(need >{padlen_needed/fs_orig:.1f}s for filtfilt with {num_taps} taps)")
        return (
            np.empty((0,0,0), dtype=np.float32),
            np.empty((0,), dtype=np.int64),
            {
                'recording_id': edf_path.stem,
                'fs': fs_orig,
                'skipped_short': True,
                'num_windows': 0,
                'num_seizure_windows': 0,
                'num_background_windows': 0,
                'num_artifact_windows_dropped': 0,
                'total_seizure_seconds_in_file': 0.0,
                'total_artifact_seconds_in_file': 0.0,
                'raw_ch_names': ch_names,
                'num_channels_raw': len(ch_names),
            },
            None,
        )

    # -------------------------------------------------
    # 0.5–40 Hz bandpass using Kaiser FIR with zero-phase filtfilt
    # -------------------------------------------------
    data_bp = apply_kaiser_bandpass(
        data,
        fs_orig,
        f_lo=0.5,
        f_hi=40.0,
        num_taps=num_taps,
        ripple_db=60.0,
    )
    print(f"after bandpass: {data_bp.shape}")

    '''
     Downsample if needed
     If fs_orig > TARGET_FS (e.g. 500 Hz), we polyphase resample down to 250 Hz
     for anti-aliasing + speed.
     If fs_orig <= TARGET_FS, we keep as is (no upsampling).
    '''
    data_ds, fs_proc = maybe_resample_to_250(
        data_bp,
        fs_orig,
        target_fs=TARGET_FS
    )
    print(f"after resample check: {data_ds.shape}, fs_proc={fs_proc}")


    # Robust z-score (median / IQR) per channel

    data_norm = robust_zscore(data_ds)
    print(f"after robust z-score: {data_norm.shape}")

    '''
    Load seizure + artifact intervals from CSV
     We will use both to decide which windows to keep.
     - seizure windows: always kept, labeled 1
     - artifact-only windows: dropped entirely
     - clean windows: kept, labeled 0
    '''
    seizure_intervals = load_seizure_intervals(ann_csv_path)
    artifact_intervals = load_artifact_intervals(ann_csv_path)

    total_sz_sec = (
        sum(e - s for (s, e) in seizure_intervals)
        if seizure_intervals else 0.0
    )
    total_artf_sec = (
        sum(e - s for (s, e) in artifact_intervals)
        if artifact_intervals else 0.0
    )

    print(f"seizure_intervals={seizure_intervals}")
    print(f"total seizure sec in file: {total_sz_sec:.2f}")
    print(f"artifact_intervals={artifact_intervals}")
    print(f"total artifact sec in file: {total_artf_sec:.2f}")

    ''' Sliding windows with artifact-aware policy
     window_data_with_artifact() returns:
       X  -> kept windows [N_keep, C_curr, T]
       y  -> labels [N_keep] (1=seizure, 0=clean background)
       t0s -> start times of windows (sec from start)
       dropped_art -> how many windows we removed due to artifact-only contamination
    '''
    X, y, t0s, dropped_art = window_data_with_artifact(
        data_norm,
        fs_proc,
        seizure_intervals,
        artifact_intervals,
        win_len_s=win_len_s,
        hop_s=hop_s,
    )

    print(
        f"=> {X.shape[0]} KEPT windows | "
        f"sz={int(np.sum(y==1))} | "
        f"bg={int(np.sum(y==0))} | "
        f"dropped_artifact={dropped_art}"
    )

    '''
    Build debug_preview for downstream reference alignment and QC plotting.
    NOTE: channel_names here are the ORIGINAL channel order for THIS EDF.
    We'll align them to the canonical montage later in build_split().
    '''
    debug_preview = None
    if X.shape[0] > 0:
        debug_preview = {
            'recording_id': edf_path.stem,
            'all_windows': X,        # [N_keep, C_curr, T]
            'all_labels': y,         # [N_keep]
            'fs': fs_proc,
            'channel_names': ch_names,
        }

  '''
     Metadata for analysis and auditing
     We'll also store:
       - total_artifact_seconds_in_file
       - num_artifact_windows_dropped
    '''
    meta = {
        'recording_id': edf_path.stem,
        'fs': fs_proc,
        'num_windows': int(len(y)),
        'num_seizure_windows': int(np.sum(y==1)),
        'num_background_windows': int(np.sum(y==0)),
        'num_artifact_windows_dropped': int(dropped_art),
        'total_seizure_seconds_in_file': total_sz_sec,
        'total_artifact_seconds_in_file': total_artf_sec,
        'skipped_short': False,
        'raw_ch_names': ch_names,
        'num_channels_raw': data_norm.shape[0],
    }

    return X, y, meta, debug_preview

## Build a split (`train`, `dev`, or `eval`)

Algorithm:
1. Collect all EDF files under the chosen split dir.
2. If `GROUP_MODE == 'subject'`, group EDFs by subject folder and cap at `MAX_SUBJECTS`.
3. For each EDF:
   - Preprocess.
   - The first usable recording defines `ref_ch_names` (our learned canonical montage order).
   - For each window in that recording (and all later recordings), align channel order to `ref_ch_names` using `align_to_reference`.
   - Compute coverage_ratio for that recording (fraction of ref channels actually present).
   - If coverage_ratio < `min_coverage` (default 0.3 = 30%), skip that recording so we don't add mostly-zero tensors.
4. Concatenate aligned windows across recordings -> one big `X_split`, `y_split`.
5. Pick the best debug candidate for QC plotting (prefer seizure-heavy recording).

In [ ]:
def _extract_subject_id(split_root: Path, edf_path: Path) -> str:
    rel = edf_path.relative_to(split_root)
    parts = rel.parts
    if len(parts) < 2:
        return "unknown"
    return parts[0]

def build_split(split_root: Path,
                group_mode=GROUP_MODE,
                max_subjects=MAX_SUBJECTS,
                max_recordings=MAX_RECORDINGS,
                win_len_s=WIN_LEN_S,
                hop_s=HOP_S,
                num_taps=NUM_TAPS,
                min_coverage=0.3):
    """
    min_coverage:
      If a recording can't map at least this fraction of the reference channels,
      we skip it (to avoid mostly-zero data).
    """

    split_root = Path(split_root)
    print(f"\nINFO ONLY ==== Processing split: {split_root.as_posix()} ====")

    edf_files = list(split_root.rglob("*.edf"))
    print(f"INFO ONLY Found {len(edf_files)} EDF files under {split_root.as_posix()}")

    X_all_list = []
    y_all_list = []
    metas = []
    debug_candidates = []

    ref_ch_names = None  # learned canonical order from the first usable recording

    def handle_edf_list(edf_iter):
        nonlocal ref_ch_names
        count_used = 0
        for edf_path in edf_iter:
            if max_recordings is not None and count_used >= max_recordings:
                break

            ann_path = find_best_annotation_for_edf(edf_path)
            if ann_path is None:
                print("SKIP NO CSV", edf_path.as_posix(),
                      "CSV candidates:", [c.name for c in edf_path.parent.glob("*.csv")])
                continue

            X_rec, y_rec, meta_rec, dbg = preprocess_single_recording_raw(
                edf_path,
                ann_path,
                win_len_s=win_len_s,
                hop_s=hop_s,
                num_taps=num_taps,
            )

            if X_rec.shape[0] == 0:
                continue

            # first usable recording defines reference channel layout
            if ref_ch_names is None:
                if dbg is None:
                    continue
                ref_ch_names = list(dbg['channel_names'])
                print(f"INFO ONLY reference channel layout set from {edf_path.stem}:")
                print(ref_ch_names)

            # align every window in this recording to ref_ch_names
            aligned_list = []
            coverages = []
            for w in X_rec:  # w: [C_curr, T]
                aligned_w, cov = align_to_reference(
                    w,
                    dbg['channel_names'],
                    ref_ch_names
                )
                aligned_list.append(aligned_w)
                coverages.append(cov)

            X_rec_aligned = np.stack(aligned_list, axis=0).astype(np.float32)
            avg_cov = float(np.mean(coverages)) if len(coverages) else 0.0
            print(f"coverage for {edf_path.stem}: {avg_cov*100:.1f}%")

            # skip low-coverage recordings (prevents mostly-zero tensors)
            if avg_cov < min_coverage:
                print(f"SKIP LOW COVERAGE {edf_path.stem} "
                      f"({avg_cov*100:.1f}% < {min_coverage*100:.1f}%)")
                continue

            # keep it
            X_all_list.append(X_rec_aligned)
            y_all_list.append(y_rec)

            meta_with_cov = dict(meta_rec)
            meta_with_cov['coverage_ratio'] = avg_cov
            metas.append(meta_with_cov)
            count_used += 1

            # candidate for QC plotting
            if dbg is not None:
                debug_candidates.append({
                    'recording_id': dbg['recording_id'],
                    'all_windows': X_rec_aligned,
                    'all_labels': y_rec,
                    'fs': dbg['fs'],
                    'channel_names': ref_ch_names,
                })

    if group_mode == "recording":
        handle_edf_list(edf_files)
    else:
        subj_to_edfs = {}
        for edf_path in edf_files:
            subj_id = _extract_subject_id(split_root, edf_path)
            subj_to_edfs.setdefault(subj_id, []).append(edf_path)

        subjects_sorted = sorted(subj_to_edfs.keys())[:max_subjects]
        print(f"INFO ONLY Subjects used (cap={max_subjects}): {subjects_sorted}")

        for subj_id in subjects_sorted:
            recs_this_subj = subj_to_edfs[subj_id]
            handle_edf_list(recs_this_subj)

    # concatenate aligned windows from all accepted recordings
    if len(X_all_list) == 0:
        X_all = np.empty((0,0,0), dtype=np.float32)
        y_all = np.empty((0,), dtype=np.int64)
    else:
        X_all = np.concatenate(X_all_list, axis=0)
        y_all = np.concatenate(y_all_list, axis=0)

    # pick best debug candidate for QC plotting (prefer more seizure windows)
    best_dbg = None
    best_sz = -1
    best_total = -1
    for dbg in debug_candidates:
        labels = dbg.get('all_labels', None)
        wins   = dbg.get('all_windows', None)
        if labels is None or wins is None:
            continue

        sz_count = int(np.sum(labels == 1))
        total_w  = int(len(labels))
        if sz_count > best_sz:
            best_dbg = dbg
            best_sz = sz_count
            best_total = total_w
        elif sz_count == best_sz and total_w > best_total:
            best_dbg = dbg
            best_total = total_w

    return X_all, y_all, metas, best_dbg

## Run preprocessing for one split

Set `SPLIT` to `'train'`, `'dev'`, or `'eval'`.

1. Call `build_split`.
2. Print dataset stats and class balance.
3. Report the debug candidate we'll use for QC plotting.

In [ ]:
SPLIT = "train"  # change to "dev" or "eval" as needed

split_dir = ROOT_DIR / SPLIT
X_split, y_split, meta_split, dbg_split = build_split(
    split_dir,
    group_mode=GROUP_MODE,
    max_subjects=MAX_SUBJECTS,
    max_recordings=MAX_RECORDINGS,
    win_len_s=WIN_LEN_S,
    hop_s=HOP_S,
    num_taps=NUM_TAPS,
    min_coverage=0.3,  # skip recordings with <30% channel match
)

print("\n=== SPLIT SUMMARY ===")
print("X_split shape:", getattr(X_split, 'shape', None))
print("y_split shape:", getattr(y_split, 'shape', None))

total_windows = len(y_split)
num_bg = int(np.sum(y_split == 0)) if total_windows > 0 else 0
num_sz = int(np.sum(y_split == 1)) if total_windows > 0 else 0
pct_bg = (100.0 * num_bg / total_windows) if total_windows > 0 else 0.0
pct_sz = (100.0 * num_sz / total_windows) if total_windows > 0 else 0.0

print(f"Total windows: {total_windows}")
print(f"Background windows: {num_bg} ({pct_bg:.2f}%)")
print(f"Seizure windows:    {num_sz} ({pct_sz:.2f}%)")
print(f"Class ratio (bg : sz) ~ {num_bg}:{num_sz}")

if dbg_split is not None:
    lbls_dbg = dbg_split.get('all_labels', None)
    if lbls_dbg is not None:
        print("Best debug recording:", dbg_split.get('recording_id','n/a'))
        print("Seizure windows in debug rec:", int(np.sum(lbls_dbg==1)))
else:
    print('WARNING: No debug candidate available to plot.')

## QC plot

We pick an informative EEG window from `dbg_split`:
1. Prefer seizure windows first (label==1), otherwise background.
2. Within that window, pick the channel with the highest std (so we avoid flat zero-fill rows).
3. Plot that channel's robust z-scored signal.
We save this as `qc_window.png` in `OUT_DIR`.

In [ ]:
def pick_informative_window(dbg_dict, motion_thresh=1e-6):
    if dbg_dict is None:
        return None, None

    X_all = dbg_dict.get('all_windows', None)   # [N_win, C_ref, T]
    y_all = dbg_dict.get('all_labels', None)    # [N_win]
    if X_all is None or y_all is None or len(y_all) == 0:
        return None, None

    seiz_idx = np.where(y_all == 1)[0]
    non_idx  = np.where(y_all == 0)[0]
    order = list(seiz_idx) + list(non_idx)

    for w_i in order:
        win = X_all[w_i]  # [C_ref, T]
        if win.size == 0:
            continue
        ch_std = np.std(win, axis=1)
        ch_i = int(np.argmax(ch_std))
        if ch_std[ch_i] > motion_thresh:
            return w_i, ch_i

    return None, None

if dbg_split is not None:
    w_i, ch_i = pick_informative_window(dbg_split, motion_thresh=1e-6)
    if w_i is not None and ch_i is not None:
        X_all = dbg_split['all_windows']
        y_all = dbg_split['all_labels']
        fs_dbg = dbg_split['fs']

        sig = X_all[w_i, ch_i, :]
        label_dbg = int(y_all[w_i])
        t_axis = np.arange(sig.shape[0]) / fs_dbg

        print(f"QC plotting window {w_i} label={label_dbg} channel {ch_i}")
        print("     std:", float(np.std(sig)),
              "min:", float(sig.min()),
              "max:", float(sig.max()))

        plt.figure(figsize=(10,4))
        plt.plot(t_axis, sig)
        plt.xlabel('Time (s)')
        plt.ylabel('Robust z-scored amplitude')
        plt.title(f'QC window w={w_i} ch={ch_i} label={label_dbg}')
        plt.grid(True)
        plt.autoscale()
        plt.tight_layout()

        qc_path = (OUT_DIR / 'qc_window.png').as_posix()
        plt.savefig(qc_path, dpi=200)
        print('INFO ONLY Saved QC plot to', qc_path)
    else:
        print('INFO ONLY Could not find any non-flat channel/window to plot.')
else:
    print('INFO ONLY Skipping QC plot; no debug recording found.')

## Save dataset + class balance

Save:
- `split_windows.npz` with:
  - `X`: all aligned windows for this split `[N_total, C_ref, T]`
  - `y`: labels `[N_total]`
  - `meta`: list of per-recording metadata dicts (coverage ratio, seizure sec, etc.)
- `class_balance.png`: simple seizure vs background bar chart.

In [ ]:
if X_split is not None and y_split is not None:
    out_npz_path = OUT_DIR / 'split_windows.npz'
    np.savez_compressed(out_npz_path.as_posix(), X=X_split, y=y_split, meta=meta_split)
    print('INFO ONLY Saved NPZ to', out_npz_path.as_posix())

    total_windows = len(y_split)
    num_bg = int(np.sum(y_split == 0)) if total_windows > 0 else 0
    num_sz = int(np.sum(y_split == 1)) if total_windows > 0 else 0

    plt.figure(figsize=(4,4))
    plt.bar(['background','seizure'], [num_bg, num_sz])
    plt.ylabel('Window count')
    plt.title('Class distribution in this split')
    plt.grid(True, axis='y')
    plt.tight_layout()

    dist_path = (OUT_DIR / 'class_balance.png').as_posix()
    plt.savefig(dist_path, dpi=200)
    print('INFO ONLY Saved class balance bar chart to', dist_path)
else:
    print('WARNING: Nothing to save / plot class balance (no data).')

In [ ]:
# Point this to the OUT_DIR
LOAD_DIR = Path("Enter Output folder here").expanduser().resolve()
npz_path = LOAD_DIR / "split_windows.npz"

print("Loading:", npz_path.as_posix())
data = np.load(npz_path, allow_pickle=True)

X = data["X"] # shape [N, C_ref, T]
y = data["y"] # shape [N]
meta_list = data["meta"] # this is an array of Python dicts (dtype=object)

print("\n=== DATASET SHAPES ===")
print("X.shape:", X.shape) # (N_win, C_ref, T)
print("y.shape:", y.shape) # (N_win,)
N, C_ref, T = X.shape if X.size > 0 else (0,0,0)
print(f"N windows = {N}")
print(f"C_ref (channels after smart canonical) = {C_ref}")
print(f"T samples per window = {T}")


# Class balance

num_bg = int(np.sum(y == 0))
num_sz = int(np.sum(y == 1))
total = len(y)

pct_bg = 100.0 * num_bg / total if total > 0 else 0.0
pct_sz = 100.0 * num_sz / total if total > 0 else 0.0

print("\n=== CLASS BALANCE ===")
print(f"background (0): {num_bg} ({pct_bg:.2f} %)")
print(f"seizure (1): {num_sz} ({pct_sz:.2f} %)")
print(f"bg : sz ratio ~ {num_bg}:{num_sz}")

'''
 Metadata summary
 Each element in meta_list corresponds to one recording:
 - 'recording_id'
 - 'fs'
 - 'num_windows'
 - 'num_seizure_windows'
 - 'num_background_windows'
 - 'total_seizure_seconds_in_file'
 - 'coverage_ratio'
 - etc.
'''

print("\n=== PER-RECORDING META (first 10 rows) ===")
for i, m in enumerate(meta_list[:10]):
    print(f"[{i}] rec_id={m.get('recording_id','?')}")
    print(f" fs_proc={m.get('fs','?')} Hz")
    print(f" windows={m.get('num_windows','?')} "
          f"seiz_win={m.get('num_seizure_windows','?')} "
          f"bg_win={m.get('num_background_windows','?')}")
    print(f" seizure_time_in_file={m.get('total_seizure_seconds_in_file','?')} sec")
    cov = m.get('coverage_ratio', None)
    if cov is not None:
        print(f" coverage={cov*100:.1f}%")
    else:
        print(" coverage=N/A (this was the reference file)")
    print("")

def plot_random_window(X, y, fs, target_label=1, motion_thresh=1e-6):
    idxs = np.where(y == target_label)[0]
    if len(idxs) == 0:
        print(f"[PLOT] No windows with label={target_label}")
        return

    w_idx = int(random.choice(idxs))
    win = X[w_idx] # shape [C_ref, T]

    ch_std = np.std(win, axis=1)
    ch_i = int(np.argmax(ch_std))
    if ch_std[ch_i] <= motion_thresh:
        print(f"[PLOT] Chosen window {w_idx} is too flat (std={ch_std[ch_i]:.2e}). Trying background instead?")
        # we could try fallback logic here, but let's just show anyway
    sig = win[ch_i, :]
    t = np.arange(sig.shape[0]) / fs

    print(f"[PLOT] window {w_idx}, label={target_label}, chosen channel idx={ch_i}, std={ch_std[ch_i]:.3f}")
    plt.figure(figsize=(10,4))
    plt.plot(t, sig)
    plt.xlabel("Time (s)")
    plt.ylabel("Robust z-scored amplitude")
    plt.title(f"Random window {w_idx} (label={target_label}) ch={ch_i}")
    plt.grid(True)
    plt.autoscale()
    plt.tight_layout()
    plt.show()

'''
 We don't store fs directly in X/y, but all aligned data in this split
 share the same fs because we either downsampled to TARGET_FS (250)
 or left lower fs alone. The metadata for any surviving recording
 will tell us. We'll just grab the first one.
'''
if len(meta_list) > 0:
    fs_est = meta_list[0].get("fs", TARGET_FS)
else:
    fs_est = TARGET_FS
print(f"\nAssumed sampling rate for plotting: {fs_est} Hz")


# Plot example seizure and background windows

plot_random_window(X, y, fs_est, target_label=1) # seizure
plot_random_window(X, y, fs_est, target_label=0) # background